In [225]:
#from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import logging
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk
import json

from gensim.models import word2vec as w2v
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /Users/bray/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [226]:
#with open('/Users/bray/Desktop/FOMC/FOMC.json', 'rb') as f:
    #data = f.readlines()
    
#data2 = json.dumps(data)
#print(data)
# remove the trailing "\n" from each line
#data = '\n'.join(map(lambda x: x.decode('utf8', 'ignore')changed_project_data['rdars']))

#data = data.decode("utf-8") 

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
#data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
#data_df = pd.read_json(data)
#print(data)

In [227]:
def tokenize_sentences(sentences):
    ret = []
    max_s = len(sentences)
    print("Got " + str(max_s) + " sentences.")
    for count, s in enumerate(sentences):
        tokens = []
        words = re.split(r'(\s+)', s)
        if len(words) > 0:
            for w in words:
                if w is not None:
                    w = w.strip()
                    w = w.lower()
                    if w.isspace() or w == "\n" or w == "\r":
                        w = None
                    if len(w) < 1:
                        w = None
                    if w is not None:
                        tokens.append(w)
        if len(tokens) > 0:
            ret.append(tokens)
        if count % 50 == 0:
            print_progress(count, max_s)
    return ret

In [228]:
def clean_sentences(tokens):
    all_stopwords = load_json("stopwords-iso.json")
    extra_stopwords = ["ssä", "lle", "h.", "oo", "on", "muk", "kov", "km", "ia", "täm", "sy", "but", ":sta", "hi", "py", "xd", "rr", "x:", "smg", "kum", "uut", "kho", "k", "04n", "vtt", "htt", "väy", "kin", "#8", "van", "tii", "lt3", "g", "ko", "ett", "mys", "tnn", "hyv", "tm", "mit", "tss", "siit", "pit", "viel", "sit", "n", "saa", "tll", "eik", "nin", "nii", "t", "tmn", "lsn", "j", "miss", "pivn", "yhn", "mik", "tn", "tt", "sek", "lis", "mist", "tehd", "sai", "l", "thn", "mm", "k", "ku", "s", "hn", "nit", "s", "no", "m", "ky", "tst", "mut", "nm", "y", "lpi", "siin", "a", "in", "ehk", "h", "e", "piv", "oy", "p", "yh", "sill", "min", "o", "va", "el", "tyn", "na", "the", "tit", "to", "iti", "tehdn", "tlt", "ois", ":", "v", "?", "!", "&","//","href","\\","``"]
    stopwords = None
    if all_stopwords is not None:
        stopwords = all_stopwords["fi"]
        stopwords += extra_stopwords
    ret = []
    max_s = len(tokens)
    for count, sentence in enumerate(tokens):
        if count % 50 == 0:
            print_progress(count, max_s)
        cleaned = []
        for token in sentence:
            if len(token) > 0:
                if stopwords is not None:
                    for s in stopwords:
                        if token == s:
                            token = None
                if token is not None:
                    if re.search("^[0-9\.\-\s\/]+$", token):
                        token = None
                if token is not None:
                    cleaned.append(token)
            if len(cleaned) > 0:
                ret.append(cleaned)
    return ret

In [229]:
def get_word_frequencies(corpus):
    frequencies = Counter()
    for sentence in corpus:
        for word in sentence:
            frequencies[word] += 1
    freq = frequencies.most_common()
    return freq


In [230]:
def try_load_or_process(filename, processor_fn, function_arg):
    load_fn = None
    save_fn = None
    if filename.endswith("json"):
        load_fn = load_json
        save_fn = save_json
    else:
        load_fn = load_bin
        save_fn = save_bin
        
    if os.path.exists(filename):
        return load_fn(filename)
    else:
        ret = processor_fn(function_arg)
        save_fn(ret, filename)
        return ret

def print_progress(current, maximum):
    sys.stdout.write("\r")
    sys.stdout.flush()
    sys.stdout.write(str(current) + "/" + str(maximum))
    sys.stdout.flush()

def save_bin(item, filename):
    with open(filename, "wb") as f:
        cPickle.dump(item, f)

def load_bin(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return cPickle.load(f)

def save_json(variable, filename):
    with io.open(filename, "w", encoding="utf-8") as f:
        f.write(unicode(json.dumps(variable, indent=4, ensure_ascii=False)))

def load_json(filename):
    ret = None
    if os.path.exists(filename):
        try:
            with io.open(filename, "r", encoding="utf-8") as f:
                ret = json.load(f)
        except:
            pass
    return ret

In [231]:
def process_raw_data(input_file):
    valid = u"0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ#@.:/ äöåÄÖÅ"
    url_match = "(https?:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]+\.?[0-9a-zA-Z\-\_]*\/?.*)"
    name_match = "\@[\_0-9a-zA-Z]+\:?"
    lines = []
    print("Loading raw data from: " + input_file)
    if os.path.exists(input_file):
        with io.open(input_file, 'r', encoding="utf-8") as f:
            lines = f.readlines()
    num_lines = len(lines)
    ret = []
    for count, text in enumerate(lines):
        if count % 50 == 0:
            print_progress(count, num_lines)
        text = re.sub(url_match, u"", text)
        text = re.sub(name_match, u"", text)
        text = re.sub("\&amp\;?", u"", text)
        text = re.sub("[\:\.]{1,}$", u"", text)
        text = re.sub("^RT\:?", u"", text)
        text = u''.join(x for x in text if x in valid)
        text = text.strip()
        if len(text.split()) > 5:
            if text not in ret:
                ret.append(text)
    return ret

In [232]:
def get_word2vec(sentences):
    num_workers = multiprocessing.cpu_count()
    num_features = 200
    epoch_count = 10
    sentence_count = len(sentences)
    w2v_file = os.path.join(save_dir, "word_vectors.w2v")
    word2vec = None
    if os.path.exists(w2v_file):
        print("w2v model loaded from " + w2v_file)
        word2vec = w2v.Word2Vec.load(w2v_file)
    else:
        word2vec = w2v.Word2Vec(sg=1,
                                seed=1,
                                workers=num_workers,
                                size=num_features,
                                min_count=6,
                                window=5,
                                sample=0)
        print("Building vocab...")
        word2vec.build_vocab(sentences)
        print("Word2Vec vocabulary length:", len(word2vec.wv.vocab))
        print("Training...")
        word2vec.train(sentences, total_examples=sentence_count, epochs=epoch_count)
        print("Saving model...")
        word2vec.save(w2v_file)
    return word2vec

In [236]:
if __name__ == '__main__':
    
    fname = 'FOMC.json'
    with open(fname) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
    sentence = []
    
    for x in content:
        nltk_tokens = nltk.word_tokenize(x)
        sentence.append(nltk_tokens)
        
        
    print("Cleaning tokens")    
    cleaned = clean_sentences(sentence)
    
    print("Getting word frequencies")
    frq = get_word_frequencies(cleaned)
    ss = len(frq)
    print("Unique words: " + str(ss))
    
    print("Instantiating word2vec model")
    word2vec = get_word2vec(sentence)
    vocab = list(word2vec.wv.vocab.keys())
    vocab_len = len(vocab)
    print("word2vec vocab contains " + str(vocab_len) + " items.")
    dim01 = word2vec.wv[vocab[0]].shape[0]
    print("word2vec items have " + str(dim01) + " features.")

Cleaning tokens
500/528Getting word frequencies
Unique words: 1063
Instantiating word2vec model
w2v model loaded from analysis/word_vectors.w2v
word2vec vocab contains 555 items.
word2vec items have 200 features.


In [237]:
import pprint as pp

In [239]:
pp.pprint(word2vec.wv.most_similar('rates', topn=10))

[('interest', 0.7389793395996094),
 ('paid', 0.7216812372207642),
 ('lower', 0.7159571051597595),
 ('establishment', 0.7094623446464539),
 ('rate', 0.7042255997657776),
 ('required', 0.699407696723938),
 ('primary', 0.6979073882102966),
 ('through', 0.6899096369743347),
 ('1/4', 0.6894129514694214),
 ('credit', 0.6891040205955505)]
